In [1]:
# IMPORTS: 
import os
import numpy as np
import mne
import scipy
import matplotlib as plt
from pd_parser.parse_pd import _read_raw, _to_tsv
import pyqtgraph as pg
import pyqtgraph as plotWidget
import readTrc
from readTrc import Trc
%matplotlib qt

data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = ''
data_path_file = 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.trc'
# data_path_file = ''
# script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
script_path = ''
# trigger_channel='Xe1'
trigger_channel=''

t_onset_s= 0
t_offset_s= -1

data_raw_file = os.path.join(data_path, data_path_file)

trc = Trc()
datX, datY, d = trc.open(data_raw_file)
plt.plot(datX, datY)
print(d)
# Or alternatively
plt.plot(trc.x, trc.y)
print(trc.d)

raw = mne.io.read_raw_edf(data_raw_file)
picks = mne.pick_channels_regexp(raw.info.ch_names, regexp=trigger_channel)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
raw.plot(order=picks,n_channels=203, event_color='cyan', scalings=3500, duration=200)
raw.get_data(picks=picks)
raw2 = raw[-1][0]
raw2 = np.array(raw2[0])

del raw

Warning, unsupported file template: in-Quick file * ... trying anyway


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 5: invalid start byte

In [5]:
import os
import neo
import numpy as np
import mne

data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = ''
data_path_file = 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.trc'

fname = os.path.join(data_path, data_path_file)


def raw_from_neo(fname):
    seg_micromed = neo.MicromedIO(filename=fname).read_segment()

    ch_names = [sig.name for sig in seg_micromed.analogsignals]

    # Because here we have the same on all chan
    sfreq = seg_micromed.analogsignals[0].sampling_rate

    data = np.asarray(seg_micromed.analogsignals)
    data *= 1e-6  # putdata from microvolts to volts
    # add stim channel
    ch_names.append('STI 014')
    data = np.vstack((data, np.zeros((1, data.shape[1]))))

    # To get sample number:
    events_time = seg_micromed.eventarrays[0].times.magnitude * sfreq
    n_events = len(events_time)
    events = np.empty([n_events, 3])
    events[:, 0] = events_time
    events[:, 2] = seg_micromed.eventarrays[0].labels.astype(int)

    ch_types = ['eog', 'eog'] + ['eeg' for _ in ch_names[2:-1]] + ['stim']
    info = mne.create_info(ch_names, sfreq, ch_types=ch_types)

    raw = mne.io.RawArray(data, info)
    raw.add_events(events)
    return raw

raw = raw_from_neo(fname)
fig = raw.plot()

events = mne.find_events(raw)
mne.viz.plot_events(events, raw.info['sfreq'])

event_id = 1
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=-0.2, tmax=0.4)
evoked = epochs.average()

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [13]:
import pandas as pd


data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = ''
data_path_file = 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.trc'

fname = os.path.join(data_path, data_path_file)

df = pd.read_fwf(fname)

In [15]:
df.tail()

,* MICROMED Brain-Quick file *   XXX XXX l                   \tz \tA J ��\t � �      ����                 ORDER �     LABCOD �   @ NOTE �D �  FLAGS `�    TRONCA ��    IMPED_B ��    IMPED_E ��    MONTAGE ��  � COMPRESS�� ,Unnamed: 1,Unnamed: 2,Unnamed: 3
3589591,"~~ ~ ~���������w���""�}Y~I~I~Y~x~�} �~�~W~...",NaN,NaN,NaN
3589592,"�}�~N}~a} |�|{|�|D~]|�|.{}""}|P{�{v{ ~!���|...",NaN,NaN,NaN
3589593,~�}}�{3{�z�~�}{}�|�}�|�}L�~ }�}�}�}�}]{W|Q|]...,NaN,NaN,NaN
3589594,|$}�~E~~�}}4}l|�{�{�{4|�}�}��}p~_}\t~9~a~�}...,NaN,NaN,NaN
3589595,~�~Q~~�}P}�|�|�| |�{�{9|�{���}�~�}}~+}�}M}�{�...,NaN,NaN,NaN
